In [1]:
#Imports
import pandas as pd
from pathlib import Path
import geopy
from geopy.geocoders import Nominatim
from geonamescache import GeonamesCache
import requests
import folium

In [2]:
#Read CSV and create df
cleandf = pd.read_csv(
    Path("tweets.csv")
)

/Users/nidalsinan/opt/anaconda3/envs/dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Drop null rows
cleandf.dropna(subset=['user_location'], inplace=True)
cleandf.head()

,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source
0,Math_Maniac,ChatGPT does ascii art\n#ChatGPT https://t.co/...,"Decatur, GA",I work with a number of tools. Right now it's ...,2020-07-20 14:24:50+00:00,467.0,1233.0,49216,False,2023-04-02 13:44:07+00:00,['ChatGPT'],Twitter Web App
1,SmashKoders,#ChatGPT is funny sometimes 😅 https://t.co/cSa...,Mumbai,My life changed while debugging code 😊,2022-06-18 04:34:18+00:00,8.0,126.0,239,False,2023-04-02 13:43:46+00:00,['ChatGPT'],Twitter Web App
2,Andreas Illenseer,Let’s put it in graphic terms. Drawing a #Unic...,"Mainz, Deutschland",Specialized in Regulatory & Digital Intelligen...,2017-10-31 19:35:24+00:00,63.0,188.0,88,False,2023-04-02 13:43:36+00:00,['Unicorn'],Twitter for iPhone
4,The Media Scoop,Italian data protection agency placed an immed...,India,Independent news aggregator & YouTuber at The ...,2023-03-07 01:51:53+00:00,0.0,37.0,0,False,2023-04-02 13:42:56+00:00,NaN,Twitter Web App
6,Simon Hodgkins,Italian minister says country's ban on ChatGPT...,Global | Ireland | USA,President | Founder | CMO | Editor in Chief,2010-10-17 07:05:46+00:00,6364.0,296.0,23528,False,2023-04-02 13:42:49+00:00,NaN,Twitter for iPhone


In [4]:
# Read the original CSV file into a pandas DataFrame

# Define a dictionary mapping each continent to a list of its countries
continent_dict = {
    'Asia': ['Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan', 'British Indian Ocean Territory', 'Brunei', 'Cambodia', 'China', 'Cyprus', 'Egypt', 'Georgia', 'Hong Kong', 'China', 'India', 'Indonesia', 'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan',  'Kuwait', 'Kyrgyzstan', 'Laos', 'Lebanon', 'Macau', 'Malaysia', 'Maldives', 'Mongolia', 'Myanmar', 'Burma', 'Nepal', 'North Korea',  'Korea', 'Oman', 'Pakistan', 'Palestine', 'Philippines', 'Qatar', 'Russia', 'Saudi Arabia', 'Singapore', 'South Korea', 'Republic of Korea', 'Sri Lanka', 'Syria', 'Taiwan', 'Tajikistan', 'Thailand', 'Timor-Leste', 'East Timor', 'Turkmenistan', 'United Arab Emirates', 'Uzbekistan', 'Vietnam', 'Yemen'],
    'Europe': ['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czechia', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland', 'Italy', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City'],
    'North America': ['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Canada', 'Costa Rica', 'Cuba', 'Dominican Dominican', 'Republic El Salvador', 'Grenada', 'Guatemala', 'Haiti', 'Honduras', 'Jamaica', 'Mexico', 'Nicaragua', 'Panama', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'Trinidad and Tobago', 'United States of America'],
    'South America': ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru', 'Suriname', 'Uruguay', 'Venezuela'],
    'Africa': ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cape Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Congo', 'Republic of the Congo', 'Democratic Republic of the Congo', 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Swaziland', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Ivory Coast', 'Republic of Côte dIvoire',  'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles','Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe'],
    'Oceania': ['Australia', 'Fiji', 'Kiribati', 'Marshall Islands', 'Micronesia', 'Nauru', 'New Zealand', 'Palau', 'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu']
}

# Define a function to map each country to its corresponding continent
def map_location_to_continent(location):
    for continent, countries in continent_dict.items():
        if location in countries:
            return continent
    return 'Other'

# Create a new column 'Continent' by applying the mapping function to the 'user_location' column
cleandf['Continent'] = cleandf['user_location'].apply(map_location_to_continent)

# Save the modified DataFrame back to the CSV file

cleandf.head()


,user_name,text,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,hashtags,source,Continent
0,Math_Maniac,ChatGPT does ascii art\n#ChatGPT https://t.co/...,"Decatur, GA",I work with a number of tools. Right now it's ...,2020-07-20 14:24:50+00:00,467.0,1233.0,49216,False,2023-04-02 13:44:07+00:00,['ChatGPT'],Twitter Web App,Other
1,SmashKoders,#ChatGPT is funny sometimes 😅 https://t.co/cSa...,Mumbai,My life changed while debugging code 😊,2022-06-18 04:34:18+00:00,8.0,126.0,239,False,2023-04-02 13:43:46+00:00,['ChatGPT'],Twitter Web App,Other
2,Andreas Illenseer,Let’s put it in graphic terms. Drawing a #Unic...,"Mainz, Deutschland",Specialized in Regulatory & Digital Intelligen...,2017-10-31 19:35:24+00:00,63.0,188.0,88,False,2023-04-02 13:43:36+00:00,['Unicorn'],Twitter for iPhone,Other
4,The Media Scoop,Italian data protection agency placed an immed...,India,Independent news aggregator & YouTuber at The ...,2023-03-07 01:51:53+00:00,0.0,37.0,0,False,2023-04-02 13:42:56+00:00,NaN,Twitter Web App,Asia
6,Simon Hodgkins,Italian minister says country's ban on ChatGPT...,Global | Ireland | USA,President | Founder | CMO | Editor in Chief,2010-10-17 07:05:46+00:00,6364.0,296.0,23528,False,2023-04-02 13:42:49+00:00,NaN,Twitter for iPhone,Other


In [5]:
#Count of continent
continent_counts = cleandf['Continent'].value_counts(dropna=False)
print(continent_counts)

Other            258306
Asia              10759
Europe             6153
Africa             1974
North America      1661
Oceania            1132
South America       988
Name: Continent, dtype: int64


In [19]:
#Map of tweets by location

world_map = folium.Map()

folium.Marker(
    location=[45.4507, 68.8319],
    popup='Asia - 10759',
    icon=folium.Icon(icon='info-sign', icon_color='Blue')
).add_to(world_map)

folium.Marker(
    location=[48.8566, 2.3522],
    popup='Europe - 6153',
    icon=folium.Icon(icon='info-sign', icon_color='Blue')
).add_to(world_map)

folium.Marker(
    location=[8.7832, 34.5085],
    popup='Africa - 1974',
    icon=folium.Icon(icon='info-sign', icon_color='Blue')
).add_to(world_map)

folium.Marker(
    location=[-20.0241, -63.5248],
    popup='South America - 988',
    icon=folium.Icon(icon='info-sign', icon_color='Blue')
).add_to(world_map)

folium.Marker(
    location=[39.7392, -104.9903],
    popup='North America - 1661',
    icon=folium.Icon(icon='info-sign', icon_color='Blue')
).add_to(world_map)

folium.Marker(
    location=[-32.92715, 151.77647],
    popup='Oceania - 1132',
    icon=folium.Icon(icon='info-sign', icon_color='blue')
).add_to(world_map)


In [20]:
world_map

In [ ]:
#Sentiment Analysis
